## Linking a dataset of real historical persons with Deterrministic Rules

While Splink is primarily a tool for probabilistic records linkage, there is functionality to perform deterministic (i.e. rules based) linkage.

In this example, we deduplicate a more realistic dataset.  The data is based on historical persons scraped from wikidata.  Duplicate records are introduced with a variety of errors introduced. The probabilistic dedupe of the same dataset can be found at `Deduplicate 50k rows historical persons`.

In [1]:
from splink.duckdb.linker import DuckDBLinker
import altair as alt
alt.renderers.enable('html')

import pandas as pd 
pd.options.display.max_rows = 1000
df = pd.read_parquet("../../data/historical_figures_with_errors_50k.parquet")
df.head()

unique_id   cluster                                         full_name  \
0  Q2296770-1  Q2296770  thomas clifford, 1st baron clifford of chudleigh   
1  Q2296770-2  Q2296770                               thomas of chudleigh   
2  Q2296770-3  Q2296770               tom 1st baron clifford of chudleigh   
3  Q2296770-4  Q2296770                              thomas 1st chudleigh   
4  Q2296770-5  Q2296770              thomas clifford, 1st baron chudleigh   

  first_and_surname first_name    surname         dob birth_place  \
0  thomas chudleigh     thomas  chudleigh  1630-08-01       devon   
1  thomas chudleigh     thomas  chudleigh  1630-08-01       devon   
2     tom chudleigh        tom  chudleigh  1630-08-01       devon   
3  thomas chudleigh     thomas  chudleigh  1630-08-01       devon   
4  thomas chudleigh     thomas  chudleigh  1630-08-01       devon   

  postcode_fake gender  occupation  
0      tq13 8df   male  politician  
1      tq13 8df   male  politician  
2      tq13 8df   male  politician  
3      tq13 8hu   None  politician  
4      tq13 8df   None  politician

When defining the settings object, simply pass your deterministic rules into `blocking_rules_to_generate_predictions`. 

For a deterministic linkage, the entire linkage methodology is based on these rules, so there is no need to define `comparisons` nor any other parameters required for model training in a probabilistic model.

In [2]:
# Simple settings dictionary will be used for exploratory analysis
settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name and l.surname = r.surname and l.dob = r.dob",
        "l.surname = r.surname and l.dob = r.dob and l.postcode_fake = r.postcode_fake",
        "l.first_name = r.first_name and l.dob = r.dob and l.occupation = r.occupation",
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
}
linker = DuckDBLinker(df, settings)

linker.debug_mode = False

Once the `linker` object is defined, you can profile the dataset columns.

In [3]:
linker.profile_columns(
    ["first_name", "surname", "substr(dob, 1,4)"], top_n=10, bottom_n=5
)

alt.VConcatChart(...)

In a deterministic linkage, the blocking rules chart shows how many records have been matched by each of the deterministic rules.

In [4]:
linker.cumulative_num_comparisons_from_blocking_rules_chart()

alt.Chart(...)

The results of the linkage can be viewed with the `deterministic_link` function.

In [5]:
df_predict = linker.deterministic_link()
df_predict.as_pandas_dataframe().head()

unique_id_l unique_id_r postcode_fake_l postcode_fake_r first_name_l  \
0  Q2296770-1  Q2296770-6        tq13 8df        tq13 8df       thomas   
1  Q2296770-2  Q2296770-6        tq13 8df        tq13 8df       thomas   
2  Q2296770-3  Q2296770-7        tq13 8df        tq13 8df          tom   
3  Q2296770-4  Q2296770-6        tq13 8hu        tq13 8df       thomas   
4  Q2296770-5  Q2296770-6        tq13 8df        tq13 8df       thomas   

  first_name_r       dob_l       dob_r  surname_l  surname_r occupation_l  \
0       thomas  1630-08-01  1630-08-01  chudleigh  chudleigh   politician   
1       thomas  1630-08-01  1630-08-01  chudleigh  chudleigh   politician   
2          tom  1630-08-01  1630-08-01  chudleigh  chudleigh   politician   
3       thomas  1630-08-01  1630-08-01  chudleigh  chudleigh   politician   
4       thomas  1630-08-01  1630-08-01  chudleigh  chudleigh   politician   

  occupation_r match_key  match_probability  
0   politician         0                1.0  
1   politician         0                1.0  
2          NaN         0                1.0  
3   politician         0                1.0  
4   politician         0                1.0

Which can be used to generate clusters. 

Note, for deterministic linkage, each comparison has been assigned a match probability of 1, so to generate clusters, set `threshold_match_probability=1` in the `cluster_pairwise_predictions_at_threshold` function.

In [6]:
clusters = linker.cluster_pairwise_predictions_at_threshold(df_predict, threshold_match_probability=1)

Completed iteration 1, root rows count 94
Completed iteration 2, root rows count 10
Completed iteration 3, root rows count 0


In [7]:
clusters.as_pandas_dataframe(limit=5)

cluster_id    unique_id   cluster                full_name  \
0  Q8006702-14  Q8006702-14  Q8006702          william chappel   
1   Q8006702-9   Q8006702-9  Q8006702         william chappell   
2   Q8007690-7   Q8007690-7  Q8007690              william day   
3   Q8008118-1   Q8008118-2  Q8008118         william duncombe   
4   Q8009468-1   Q8009468-5  Q8009468  william fuller-maitland   

         first_and_surname first_name          surname         dob  \
0          william chappel    william          chappel         NaN   
1         william chappell    william         chappell         NaN   
2              william day    william              day  1529-11-01   
3         william duncombe    william         duncombe  1730-01-01   
4  william fuller-maitland    william  fuller-maitland  1844-05-06   

  birth_place postcode_fake gender occupation  
0   quedgeley           NaN   male        NaN  
1  gloucester       gl2 3qe   male        NaN  
2         NaN       sy5 7pq   male        NaN  
3    kirklees      wf14 8ly   male        NaN  
4     silsden       bb1 3es   male        NaN

These results can then be passed into the `Cluster Studio Dashboard`.

In [8]:
linker.cluster_studio_dashboard(df_predict, clusters, "50k_deterministic_cluster.html", sampling_method='by_cluster_size', overwrite=True)

from IPython.display import IFrame

IFrame(
    src="./50k_deterministic_cluster.html", width="100%", height=1200
)  